In [1]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets
from tqdm import tqdm

from pixelsnail import PixelSNAIL


def train(epoch, loader, model, optimizer, device):
    loader = tqdm(loader)

    criterion = nn.CrossEntropyLoss()

    for i, (img, label) in enumerate(loader):
        model.zero_grad()

        img = img.to(device)

        out, _ = model(img)
        loss = criterion(out, img)
        loss.backward()

        optimizer.step()

        _, pred = out.max(1)
        correct = (pred == img).float()
        accuracy = correct.sum() / img.numel()

        loader.set_description(
            (f'epoch: {epoch + 1}; loss: {loss.item():.5f}; ' f'acc: {accuracy:.5f}')
        )


class PixelTransform:
    def __init__(self):
        pass

    def __call__(self, input):
        ar = np.array(input)

        return torch.from_numpy(ar).long()




  0%|          | 0/1875 [00:00<?, ?it/s]

test


epoch: 1; loss: 85.86835; acc: 0.81262:   0%|          | 3/1875 [00:03<28:32,  1.09it/s] 

test
test


epoch: 1; loss: 11.91310; acc: 0.29050:   0%|          | 5/1875 [00:04<13:52,  2.25it/s]

test
test
test


epoch: 1; loss: 11.91310; acc: 0.29050:   0%|          | 5/1875 [00:04<26:33,  1.17it/s]


KeyboardInterrupt: 

In [ ]:
    device = 'cuda'
    epoch = 10

    dataset = datasets.MNIST('.', transform=PixelTransform(), download=True)
    loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

    model = PixelSNAIL([28, 28], 256, 128, 5, 2, 4, 128)
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for i in range(10):
        train(i, loader, model, optimizer, device)
        torch.save(model.state_dict(), f'mnist_{str(i + 1).zfill(3)}.pt')

In [ ]:
import torch
import torch.nn.functional as F

from pixelsnail import PixelSNAIL

def sample_pixel(model, canvas, x, y):
    # Condition the model on the already sampled pixels (left and above)
    condition_input = canvas[:, :, :x, :y]
    
    # Sample the pixel value for the current position
    with torch.no_grad():
        print(model(condition_input))
        logits, _ = model(condition_input)
        pixel_probabilities = F.softmax(logits[:, :, x, y], dim=-1)
        pixel_value = torch.multinomial(pixel_probabilities, 1).squeeze()
    
    return pixel_value

def generate_image(model, canvas):
    batch_size, channel, height, width = canvas.shape
    
    for x in range(height):
        for y in range(width):
            pixel_value = sample_pixel(model, canvas, x, y)
            canvas[:, :, x, y] = pixel_value
    
    return canvas

# Define the shape of the image and number of classes
image_shape = (32,1, 28, 28)  # (batch_size, channels, height, width)
num_classes = 256  # Assuming you have 256 possible pixel values

# Initialize an empty canvas
canvas = torch.zeros(image_shape)

#



# Load your trained PixelSNAIL model
# trained_model = ...

# load trained model from file
model = PixelSNAIL([28, 28], 256, 128, 5, 2, 4, 128)
model.load_state_dict(torch.load('mnist_010.pt'))


# Generate an image
generated_image = generate_image(model, canvas)

# You can convert the generated_image tensor to a numpy array and visualize it as an image
# generated_image_np = generated_image.numpy()
# ...

# Optionally, you can save the generated image to a file
# torch.save(generated_image, "generated_image.pth")

ValueError: too many values to unpack (expected 3)